<a href="https://colab.research.google.com/github/amitgupta911/EndOfDays/blob/master/SimpleParaphrasing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## import libraries 
from keras.layers import Dense,LSTM,Embedding,GRU,Input
from keras.models import Sequential,Model
from keras.preprocessing.text import Tokenizer
import numpy as np
import keras.utils as KU
import traceback
import numpy as np

Using TensorFlow backend.


In [0]:
##read dataset
with open('../content/quora_duplicate_questions.tsv') as f:
  data=f.read().split('\n')

FileNotFoundError: ignored

In [0]:
data[1].split('\t')[5]

'0'

In [0]:
corpus=[]
for i in data:
  line=i.split('\t')
  if(i!= " " and len(line)>5):
    is_duplicate=line[5]
    if(is_duplicate=='1'):
        corpus.append(i)
    
  
  
    

In [0]:
corpus

["5\t11\t12\tAstrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?\tI'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?\t1",
 '7\t15\t16\tHow can I be a good geologist?\tWhat should I do to be a great geologist?\t1',
 '11\t23\t24\tHow do I read and find my YouTube comments?\tHow can I see all my Youtube comments?\t1',
 '12\t25\t26\tWhat can make Physics easy to learn?\tHow can you make physics easy to learn?\t1',
 '13\t27\t28\tWhat was your first sexual experience like?\tWhat was your first sexual experience?\t1',
 '15\t31\t32\tWhat would a Trump presidency mean for current international master’s students on an F1 visa?\tHow will a Trump presidency affect the students presently in US or planning to study in US?\t1',
 '16\t33\t34\tWhat does manipulation mean?\tWhat does manipulation means?\t1',
 '18\t37\t38\tWhy are so many Quora users posting questions that are readily answered on Google?\tWhy do people ask Quora q

In [0]:
len(corpus)

149263

In [0]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()

In [0]:
## Not create input text as English and output text as Hindi
try:
  for line in corpus[:10000]:
      ##print(line)
      if(line !=""):
          text=line.split('\t')
          input_text=text[3]
          target_text=text[4]
          input_texts.append(input_text)
          target_text='\t'+target_text+'\n'
          ##print(target_text)
          target_texts.append(target_text)
          for char in input_text:
              if char not in input_characters:
                  input_characters.add(char)
              
          for char in target_text:
              if char not in target_characters:
                  target_characters.add(char)
        
     
except Exception as e:
  print(e)
  traceback.print_exc()

In [0]:
len(input_texts)

10000

In [0]:
len(target_texts)

10000

In [0]:
## lets do some more preprocesing on input and target text

input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)

max_encoder_seq_length=max([ len(i) for i in input_texts])
max_decoder_seq_length=max([ len(i) for i in target_texts])


print("Number of  encoded tokens are ",len(input_characters))
print("Number of  decoded tokens are ",len(target_characters))

print("Maximum length of sequence in Input sequences is",max_encoder_seq_length)
print("Maximum length of sequence in Output sequences is", max_decoder_seq_length)

Number of  encoded tokens are  123
Number of  decoded tokens are  121
Maximum length of sequence in Input sequences is 220
Maximum length of sequence in Output sequences is 262


In [0]:
## create a dictionary set for every sententces word and it index
input_word_index=dict([(char,i) for i,char in enumerate(input_characters)])
target_word_index=dict([(char,i) for i,char in enumerate(target_characters)])

In [0]:
## Now we need 3 types of data which includes like input data ,target-data and target state from encoder

## so lets create it

encoder_input_data=np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

In [0]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(10000, 220, 123)
(10000, 262, 121)
(10000, 262, 121)


In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_word_index[char]] = 1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_word_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_word_index[char]] = 1.

In [0]:
## define some basic configs
batch_size = 64  # batch size for training
epochs = 100  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space

In [0]:
##Define Encoder Architecture

encoder_input=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
_, state_h, state_c = encoder(encoder_input)
encoder_states = [state_h, state_c]

W0708 11:26:39.110043 140071105329024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0708 11:26:39.158410 140071105329024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0708 11:26:39.181778 140071105329024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [0]:
##Not lets Define Decoder Architecture

Decoder_input=Input(shape=(None,num_decoder_tokens))
decoder=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_ = decoder(Decoder_input,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
model=Model(inputs=[encoder_input,Decoder_input],outputs=decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 123)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 121)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 389120      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  387072      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [0]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=100,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 427s 53ms/step - loss: 0.5955 - acc: 0.0442 - val_loss: 0.5485 - val_acc: 0.0574
Epoch 2/100
8000/8000 [==============================] - 425s 53ms/step - loss: 0.4987 - acc: 0.0702 - val_loss: 0.4640 - val_acc: 0.0765
Epoch 3/100
8000/8000 [==============================] - 427s 53ms/step - loss: 0.4464 - acc: 0.0804 - val_loss: 0.4309 - val_acc: 0.0806
Epoch 4/100
8000/8000 [==============================] - 425s 53ms/step - loss: 0.4164 - acc: 0.0872 - val_loss: 0.4024 - val_acc: 0.0896
Epoch 5/100
8000/8000 [==============================] - 425s 53ms/step - loss: 0.3940 - acc: 0.0935 - val_loss: 0.3846 - val_acc: 0.0947
Epoch 6/100
8000/8000 [==============================] - 426s 53ms/step - loss: 0.3760 - acc: 0.0985 - val_loss: 0.3700 - val_acc: 0.0996
Epoch 7/100
8000/8000 [==============================] - 425s 53ms/step - loss: 0.3607 - acc: 0.1029 - val_loss: 0.3569 - va

In [0]:
model.save('simpleparaphrasing.h5')

NameError: ignored